In [1]:
import scipy.stats as st
import numpy as np
import sys

from random import random
from subst_mat_lib_2 import *
from rand_arr_generate import *
from math import log10

<b>Loading substitution matrices</b>
<p> Substitution matrices were obtained with the substitution_matrix_r4.py script</p>
<p> Using the subst_mat_read function from the subst_mat_lib library</p>
<p> Loading separately: <br>
    1. Matrices for synonimous sites <br>
    2. Matrices for nonsynonimous sites <br>
    3. Matrices for both synonimous and nonsynonimous sites
</p>

In [2]:
smat_bim_all = subst_mat_read("smat_bim.txt")[1]
smat_oct_all = subst_mat_read("smat_oct.txt")[1]
smat_sep_all = subst_mat_read("smat_sep.txt")[1]
smat_squ_all = subst_mat_read("smat_squ.txt")[1]

smat_bim_syn = subst_mat_read("smat_bim_syn.txt")[1]
smat_oct_syn = subst_mat_read("smat_oct_syn.txt")[1]
smat_sep_syn = subst_mat_read("smat_sep_syn.txt")[1]
smat_squ_syn = subst_mat_read("smat_squ_syn.txt")[1]

smat_bim_nsyn = subst_mat_read("smat_bim_nsyn.txt")[1]
smat_oct_nsyn = subst_mat_read("smat_oct_nsyn.txt")[1]
smat_sep_nsyn = subst_mat_read("smat_sep_nsyn.txt")[1]
smat_squ_nsyn = subst_mat_read("smat_squ_nsyn.txt")[1]

<b>Calculating sums of matrices for closely related species</b>
<p>Using the sum_subst_mats function from the subst_mat_lib library

In [3]:
smat_squ_sep_all = sum_subst_mats(smat_squ_all, smat_sep_all)
smat_bim_oct_all = sum_subst_mats(smat_bim_all, smat_oct_all)

smat_bim_oct_syn = sum_subst_mats(smat_bim_syn, smat_oct_syn)
smat_squ_sep_syn = sum_subst_mats(smat_squ_syn, smat_sep_syn)

smat_bim_oct_nsyn = sum_subst_mats(smat_bim_nsyn, smat_oct_nsyn)
smat_squ_sep_nsyn = sum_subst_mats(smat_squ_nsyn, smat_sep_nsyn)

<h3>Q<sub>→*</sub> count</h3>

<h4>And estimating different effects, underlying Q<sub>→*</sub></h4>
<p>Q<sub>→*</sub> harbours two effects: <br>
1. Change of the E → G mutation frequency (relative to A → G)<br>
2. Change of the E → Y mutation frequency (relative to A → Y)<br>
Thus, we consider four mutatioal probabilities:
E → G, E → Y, A → G, A → Y</p>
<p>When counting R and Q for synonimous and non-synonimous sites separately,
we employ coefficients reflecting differences in synonimous and non-synonimous
mutation rates</p>

In [4]:
def R_and_Q_E2G_count(smat, precision, alpha_arr, coeff_G, coeff_Y):
    let_arr = ['A','E','G','C','T']
    sum_E = colsum(smat, 'E')
    p_E2G = float(smat['E']['G'])/sum_E
    p_E2Y = float(smat['E']['C'] + smat['E']['T'])/sum_E
    sum_A = colsum(smat, 'A')
    p_A2G = float(smat['A']['G'])/sum_A
    p_A2Y = float(smat['A']['C'] + smat['A']['T'])/sum_A
    
    mean_rg = p_E2G*coeff_G/p_A2G
    mean_ry = p_E2Y*coeff_Y/p_A2Y
    mean_q = mean_rg/mean_ry
#    print(mean_rg)
#    print(mean_ry)
#    print(mean_q)
#    
    E2G = rand_arr_generate(precision, sum_E, p_E2G)
    E2Y = rand_arr_generate(precision, sum_E, p_E2Y)
    A2G = rand_arr_generate(precision, sum_A, p_A2G)
    A2Y = rand_arr_generate(precision, sum_A, p_A2Y)
    
    for i in range(precision):
        if E2Y[i] == 0:
            E2Y[i] = 0.001
        if A2G[i] == 0:
            A2G[i] = 0.001
        if A2Y[i] == 0:
            A2Y[i] = 0.001
    
    rg_vals = [E2G[i]*coeff_G*sum_A/(A2G[i]*sum_E) for i in range(precision)]
    ry_vals = [E2Y[i]*coeff_Y*sum_A/(A2Y[i]*sum_E) for i in range(precision)]
    q_vals = [rg_vals[i]/ry_vals[i] for i in range(precision)]
    rg_vals = sorted(rg_vals)
    ry_vals = sorted(ry_vals)
    q_vals = sorted(q_vals)
    
    alpha_dict_rg = dict()
    alpha_dict_ry = dict()
    alpha_dict_q = dict()
    for alpha in alpha_arr:
        alpha_dict_q[alpha] = dict()
        alpha_dict_q[alpha]["min"] = q_vals[int(precision*(alpha/2))]
        alpha_dict_q[alpha]["max"] = q_vals[-int(precision*(alpha/2))]
        alpha_dict_rg[alpha] = dict()
        alpha_dict_rg[alpha]["min"] = rg_vals[int(precision*(alpha/2))]
        alpha_dict_rg[alpha]["max"] = rg_vals[-int(precision*(alpha/2))]
        alpha_dict_ry[alpha] = dict()
        alpha_dict_ry[alpha]["min"] = ry_vals[int(precision*(alpha/2))]
        alpha_dict_ry[alpha]["max"] = ry_vals[-int(precision*(alpha/2))]
        
    return mean_q, mean_rg, mean_ry, alpha_dict_rg, alpha_dict_ry, alpha_dict_q

def alpha_dict_print(alpha_dict):
    res_arr = []
    for alpha in sorted(alpha_dict.keys())[::-1]:
        res_arr.append(alpha_dict[alpha]["min"])
        res_arr.append(alpha_dict[alpha]["max"])
    return res_arr

def print_for_smat(smat, precision, alpha_arr, species, outfile, coeff_G, coeff_Y):
    mean_q, mean_rg, mean_ry, alpha_dict_rg, alpha_dict_ry, alpha_dict_q = R_and_Q_E2G_count(smat, precision, alpha_arr, coeff_G, coeff_Y)
    alpha_arr_rg = alpha_dict_print(alpha_dict_rg)
    alpha_arr_ry = alpha_dict_print(alpha_dict_ry)
    alpha_arr_q = alpha_dict_print(alpha_dict_q)
    outfile.write('Q' + species + '\t' + str(mean_q)+'\t'+'\t'.join(map(str,alpha_arr_q))  + '\n')
    outfile.write('RG' + species + '\t' + str(mean_rg)+'\t'+'\t'.join(map(str,alpha_arr_rg)) + '\n')
    outfile.write('RY' + species + '\t' + str(mean_ry)+'\t'+'\t'.join(map(str,alpha_arr_ry)) + '\n')

In [5]:
#Coeffs: substitution_matrix_6_1.py output
#subst_mat_print(smat_squ_sep_all)
coeff_G_all = 1.0
coeff_Y_all = 1.0
coeff_G_syn = 0.0198403586868/0.0839480367971
coeff_Y_syn = 0.0103003952764/0.0342816172271
coeff_G_nsyn = 0.0198403586868/0.00349503011675
coeff_Y_nsyn = 0.0103003952764/0.00418598021962

outfile_sep_squ_all = open("rs_and_qs/sep_squ_all_1.txt", 'w')
outfile_sep_squ_nsyn = open("rs_and_qs/sep_squ_nsyn_1.txt", 'w')
outfile_sep_squ_syn = open("rs_and_qs/sep_squ_syn_1.txt", 'w')

print_for_smat(smat_squ_sep_all,  100000, [0.1, 0.05, 0.01], "Q2*", outfile_sep_squ_all , coeff_G_all , coeff_Y_all )
print_for_smat(smat_squ_sep_nsyn, 100000, [0.1, 0.05, 0.01], "Q2*", outfile_sep_squ_nsyn, coeff_G_nsyn, coeff_Y_nsyn)
print_for_smat(smat_squ_sep_syn,  100000, [0.1, 0.05, 0.01], "Q2*", outfile_sep_squ_syn , coeff_G_syn , coeff_Y_syn )

#print_for_smat(smat_squ_sep_all,  100000, [0.1, 0.05, 0.01], "Q2*", sys.stdout, coeff_G_all, coeff_Y_all)
#print_for_smat(smat_squ_sep_nsyn, 100000, [0.1, 0.05, 0.01], "Q2*", sys.stdout, coeff_G_nsyn, coeff_Y_nsyn)
#print_for_smat(smat_squ_sep_syn,  100000, [0.1, 0.05, 0.01], "Q2*", sys.stdout, coeff_G_syn, coeff_Y_syn)

outfile_sep_squ_all.close()
outfile_sep_squ_nsyn.close()
outfile_sep_squ_syn.close()

<h3>Q<sub>*→</sub> count</h3>

<h4>Estimating different effects, underlying Q<sub>*→</sub></h4>
<p>This Q harbours two effects: <br>
1. Change of the G → E mutation frequency (relative to G → A)<br>
2. Change of the Y → E mutation frequency (relative to Y → A)<br>
Thus, we consider four mutatioal probabilities:
G → E, Y → E, G → A, Y → A</p>
<p>But we consider conditional probabilities, that take into account drastic differences beween numbers of E and A sites</p>

In [46]:
def R_and_Q_G2E_count(smat, precision, alpha_arr, cond_prob_E, cond_prob_A, coeff_G, coeff_Y):
    sum_G = colsum(smat, 'G')
    p_G2E = float(smat['G']['E'])/sum_G
    p_G2A = float(smat['G']['A'])/sum_G
    sum_Y = colsum(smat, 'C') + colsum(smat, 'T')
    p_Y2E = float(smat['C']['E'] + smat['T']['E'])/sum_Y
    p_Y2A = float(smat['C']['A'] + smat['T']['A'])/sum_Y
    
    mean_rg = p_G2E*cond_prob_A*coeff_G/(p_G2A*cond_prob_E)
    mean_ry = p_Y2E*cond_prob_A*coeff_Y/(p_Y2A*cond_prob_E)
    mean_q = mean_rg/mean_ry
    
    G2E = rand_arr_generate(precision, sum_G, p_G2E)
    Y2E = rand_arr_generate(precision, sum_Y, p_Y2E)
    G2A = rand_arr_generate(precision, sum_G, p_G2A)
    Y2A = rand_arr_generate(precision, sum_Y, p_Y2A)
    
    for i in range(precision):
        if Y2E[i] == 0:
            Y2E[i] = 0.001
        if G2A[i] == 0:
            G2A[i] = 0.001
        if Y2A[i] == 0:
            Y2A[i] = 0.001

    rg_vals = [G2E[i]*cond_prob_A*coeff_G/(G2A[i]*cond_prob_E) for i in range(precision)]
    ry_vals = [Y2E[i]*cond_prob_A*coeff_Y/(Y2A[i]*cond_prob_E) for i in range(precision)]
    q_vals = [rg_vals[i]/ry_vals[i] for i in range(precision)]
    rg_vals = sorted(rg_vals)
    ry_vals = sorted(ry_vals)
    q_vals = sorted(q_vals)
    
    alpha_dict_rg = dict()
    alpha_dict_ry = dict()
    alpha_dict_q = dict()
    for alpha in alpha_arr:
        alpha_dict_q[alpha] = dict()
        alpha_dict_q[alpha]["min"] = q_vals[int(precision*(alpha/2))]
        alpha_dict_q[alpha]["max"] = q_vals[-int(precision*(alpha/2))]
        alpha_dict_rg[alpha] = dict()
        alpha_dict_rg[alpha]["min"] = rg_vals[int(precision*(alpha/2))]
        alpha_dict_rg[alpha]["max"] = rg_vals[-int(precision*(alpha/2))]
        alpha_dict_ry[alpha] = dict()
        alpha_dict_ry[alpha]["min"] = ry_vals[int(precision*(alpha/2))]
        alpha_dict_ry[alpha]["max"] = ry_vals[-int(precision*(alpha/2))]
        
    return mean_q, mean_rg, mean_ry, alpha_dict_rg, alpha_dict_ry, alpha_dict_q

def print_for_smat_2(smat, precision, alpha_arr, cond_prob_E, cond_prob_A, coeff_G, coeff_Y, species, outfile):
    mean_q, mean_rg, mean_ry, alpha_dict_rg, alpha_dict_ry, alpha_dict_q = R_and_Q_G2E_count(smat, precision, alpha_arr, cond_prob_E, cond_prob_A, coeff_G, coeff_Y)
    alpha_arr_rg = alpha_dict_print(alpha_dict_rg)
    alpha_arr_ry = alpha_dict_print(alpha_dict_ry)
    alpha_arr_q = alpha_dict_print(alpha_dict_q)
    outfile.write('Q' + species + '\t' + str(mean_q)+'\t'+'\t'.join(map(str,alpha_arr_q))  + '\n')
    outfile.write('RG' + species + '\t' + str(mean_rg)+'\t'+'\t'.join(map(str,alpha_arr_rg)) + '\n')
    outfile.write('RY' + species + '\t' + str(mean_ry)+'\t'+'\t'.join(map(str,alpha_arr_ry)) + '\n')

In [61]:
nf_E_all = float(126868 + 76886 + 3768 + 6089)/(7769122 + 6831947)
nf_A_all = 1 - nf_E_all

nf_E_syn = float(28688 + 44406)/(7769122 + 6831947)
nf_A_syn = 1 - nf_E_syn

nf_E_nsyn = nf_E_all - nf_E_syn
nf_A_nsyn = 1 - nf_E_nsyn

nf_E_all_oct = float(75316 + 39634 + 17414 + 5281 + 114624 + 44472 + 16751 + 3733)/(10043547 + 7749608)
nf_A_all_oct = 1 - nf_E_all_oct

nf_E_syn_oct = float(26783 + 41203)/(10043547 + 7749608)
nf_A_syn_oct = 1 - nf_E_syn_oct

nf_E_nsyn_oct = nf_E_all_oct - nf_E_syn_oct
nf_A_nsyn_oct = 1 - nf_E_nsyn_oct

coeff_G_all = 1.0
coeff_Y_all = 1.0
coeff_G_syn = 0.0198403586868/0.0839480367971
coeff_Y_syn = 0.0103003952764/0.0342816172271
coeff_G_nsyn = 0.0198403586868/0.00349503011675
coeff_Y_nsyn = 0.0103003952764/0.00418598021962

coeff_G_oct_all = 1.0
coeff_Y_oct_all = 1.0
coeff_G_oct_syn = 0.00296624439781/0.0172659591319
coeff_Y_oct_syn = 0.000798302147627/0.00185663133626
coeff_G_oct_nsyn = 0.00296624439781/0.0011175505165
coeff_Y_oct_nsyn = 0.000798302147627/0.000437030214284

outfile_sep_squ_all  = open("rs_and_qs/sep_squ_all_1.txt", 'a')
outfile_sep_squ_nsyn = open("rs_and_qs/sep_squ_nsyn_1.txt", 'a')
outfile_sep_squ_syn  = open("rs_and_qs/sep_squ_syn_1.txt", 'a')
outfile_bim_oct_all  = open("rs_and_qs/bim_oct_all_1.txt", 'w')
outfile_bim_oct_nsyn = open("rs_and_qs/bim_oct_nsyn_1.txt", 'w')
outfile_bim_oct_syn  = open("rs_and_qs/bim_oct_syn_1.txt", 'w')

#print_for_smat_2(smat_squ_sep_all,  100000, [0.1, 0.05, 0.01], nf_E_all, nf_A_all, coeff_G_all, coeff_Y_all, "Q*2", sys.stdout)
#print_for_smat_2(smat_squ_sep_nsyn, 100000, [0.1, 0.05, 0.01], nf_E_nsyn, nf_A_nsyn, coeff_G_nsyn, coeff_Y_nsyn, "Q*2", sys.stdout)
#print_for_smat_2(smat_squ_sep_syn,  100000, [0.1, 0.05, 0.01], nf_E_syn, nf_A_syn, coeff_G_syn, coeff_Y_syn, "Q*2", sys.stdout)
#
#print_for_smat_2(smat_bim_oct_all,  100000, [0.1, 0.05, 0.01], nf_E_all_oct, nf_A_all_oct, coeff_G_oct_all, coeff_Y_all, "Q*2", sys.stdout)
#print_for_smat_2(smat_bim_oct_nsyn, 100000, [0.1, 0.05, 0.01], nf_E_nsyn_oct, nf_A_nsyn_oct, coeff_G_oct_nsyn, coeff_Y_nsyn, "Q*2", sys.stdout)
#print_for_smat_2(smat_bim_oct_syn,  100000, [0.1, 0.05, 0.01], nf_E_syn_oct, nf_A_syn_oct, coeff_G_oct_syn, coeff_Y_syn, "Q*2", sys.stdout)

print_for_smat_2(smat_squ_sep_all,  100000, [0.1, 0.05, 0.01], nf_E_all, nf_A_all, coeff_G_all, coeff_Y_all, "Q*2", outfile_sep_squ_all)
print_for_smat_2(smat_squ_sep_nsyn, 100000, [0.1, 0.05, 0.01], nf_E_nsyn, nf_A_nsyn, coeff_G_nsyn, coeff_Y_nsyn, "Q*2", outfile_sep_squ_nsyn)
print_for_smat_2(smat_squ_sep_syn,  100000, [0.1, 0.05, 0.01], nf_E_syn, nf_A_syn, coeff_G_syn, coeff_Y_syn, "Q*2", outfile_sep_squ_syn)

print_for_smat_2(smat_bim_oct_all,  100000, [0.1, 0.05, 0.01], nf_E_all_oct, nf_A_all_oct, coeff_G_oct_all, coeff_Y_all, "Q*2", outfile_bim_oct_all)
print_for_smat_2(smat_bim_oct_nsyn, 100000, [0.1, 0.05, 0.01], nf_E_nsyn_oct, nf_A_nsyn_oct, coeff_G_oct_nsyn, coeff_Y_nsyn, "Q*2", outfile_bim_oct_nsyn)
print_for_smat_2(smat_bim_oct_syn,  100000, [0.1, 0.05, 0.01], nf_E_syn_oct, nf_A_syn_oct, coeff_G_oct_syn, coeff_Y_syn, "Q*2", outfile_bim_oct_syn)

outfile_sep_squ_all.close()
outfile_sep_squ_nsyn.close()
outfile_sep_squ_syn.close()
outfile_bim_oct_all.close()
outfile_bim_oct_nsyn.close()
outfile_bim_oct_syn.close()

hello


<h3>Editing events parallel evolution</h3>
<p>Data is obtained with the parallel_count_2.py script separately for synonimous, non-synonimous and all sites</p>
<p>Confidence intervals are obtained from binomial distribution</p>
<p>Parallel events A → G are considered as a control</p>

In [23]:
def confints_paral(n_muts, n_sites, ref_p_1, ref_p_2, alpha_arr, annotation, outfile):
    res_arr = [annotation]
    ref_p = float(n_muts)/n_sites
    res_arr.append(ref_p/(ref_p_1*ref_p_2))
    for alpha in alpha_arr:
        a = st.binom.interval(alpha, n_sites, ref_p, loc=0)
        res_arr.append((float(a[0])/n_sites)/(ref_p_1*ref_p_2))
        res_arr.append((float(a[1])/n_sites)/(ref_p_1*ref_p_2))
    outfile.write('\t'.join(map(str, res_arr)) + '\n')
        
    

In [27]:
n_muts_all = 83
n_muts_syn = 21
n_muts_nsyn = 61

n_muts_G = 16
n_muts_G_syn = 14
n_muts_G_nsyn = 2
syn_G_frac = 0.34
n_sites = 35494


outfile_main = open("sep_squ_fig4.txt", 'a')
outfile_nes = open("sep_squ_nes.txt", 'a')
outfile_ses = open("sep_squ_ses.txt", 'a')

#All
ref_p_1 = max(smat_squ['A']['E']/colsum(smat_squ, 'A'), smat_sep['A']['E']/colsum(smat_sep, 'A'))
ref_p_2 = max(smat_oct['A']['E']/colsum(smat_oct, 'A'), smat_bim['A']['E']/colsum(smat_bim, 'A'))

confints_paral(n_muts_all, n_sites, ref_p_1, ref_p_2, [0.9,0.95,0.99], "All_sites_E", outfile_main)

#syn
ref_p_1 = max(smat_squ_syn['A']['E']/colsum(smat_squ_syn, 'A'), smat_sep_syn['A']['E']/colsum(smat_sep_syn, 'A'))
ref_p_2 = max(smat_oct_syn['A']['E']/colsum(smat_oct_syn, 'A'), smat_bim_syn['A']['E']/colsum(smat_bim_syn, 'A'))

confints_paral(n_muts_G, n_sites, ref_p_1, ref_p_2, [0.9,0.95,0.99], "syn", outfile_ses)

#nsyn
ref_p_1 = max(smat_squ_nsyn['A']['E']/colsum(smat_squ_nsyn, 'A'), smat_sep_nsyn['A']['E']/colsum(smat_sep_nsyn, 'A'))
ref_p_2 = max(smat_oct_nsyn['A']['E']/colsum(smat_oct_nsyn, 'A'), smat_bim_nsyn['A']['E']/colsum(smat_bim_nsyn, 'A'))

confints_paral(n_muts_G, n_sites, ref_p_1, ref_p_2, [0.9,0.95,0.99], "nsyn", outfile_nes)

#G_all
ref_p_1 = max(smat_squ['A']['G']/colsum(smat_squ, 'A'), smat_sep['A']['G']/colsum(smat_sep, 'A'))
ref_p_2 = max(smat_oct['A']['G']/colsum(smat_oct, 'A'), smat_bim['A']['G']/colsum(smat_bim, 'A'))

confints_paral(n_muts_G, n_sites, ref_p_1, ref_p_2, [0.9,0.95,0.99], "G", outfile_main)

#G_syn
confints_paral(n_muts_G_syn, int(n_sites*syn_G_frac), ref_p_1, ref_p_2, [0.9,0.95,0.99], "G", outfile_ses)

#G_nsyn
confints_paral(n_muts_G_nsyn, int(n_sites*(1-syn_G_frac)), ref_p_1, ref_p_2, [0.9,0.95,0.99], "G", outfile_nes)

outfile_main.close()
outfile_nes.close()
outfile_ses.close()

<h3>Evolution modelling using substitution matrices</h3>
<p>Normalized substitution matrix is multiplied multiple times with the vector with nucleotide numbers</p>
<p>Along with that, two sets of values are calculated:<br>
1. Numbers of E nucleotides that emerge from each nucleotide (toE)<br>
2. Numbers of nucleotides that emerge from E nucleotides (fromE)</p>

In [22]:
def print_evol(smat_n, nucl_arr, n_range, outfile_name, outf_fromE_name, outf_toE_name):
#    let_arr = ['A','G','C','T']
    let_arr = ['A','E','G','C','T']
    a = np.array([[smat_n[l1][l2] for l2 in let_arr] for l1 in let_arr])
    b = np.array(nucl_arr)
    with open(outfile_name, "w") as outfile_evol, open(outf_fromE_name, 'w') as outf_fromE, open(outf_toE_name, 'w') as outf_toE:
        
        outfile_evol.write('\t'.join(let_arr) + '\n')
        outfile_evol.write('\t'.join(map(str, b)) + '\n')
        
        outf_fromE.write('\t'.join(let_arr) + '\n')
        outf_toE.write('\t'.join(let_arr) + '\n')
        for i in range(n_range):
            b1 = np.matmul(b, a)
            outfile_evol.write('\t'.join(map(str,b1)) + '\t' + str(sum(b1)) + '\n')
            
            fromE_arr = []
            toE_arr = []
            for i in range(5):
                fromE_arr.append(b[1]*smat_n['E'][let_arr[i]])
                toE_arr.append(b[i]*smat_n[let_arr[i]]['E'])
                
            outf_fromE.write('\t'.join(map(str,fromE_arr)) + '\n')
            outf_toE.write('\t'.join(map(str, toE_arr)) + '\n')
            b = b1

In [39]:
nucl_arr_bim = [9968231, 76862, 6211485, 6099841, 9087468]
nucl_arr_oct = [7634984, 117842, 4770934, 4729830, 6999038]
nucl_arr_sep = [7642254, 130636, 4896006, 4811269, 6990703]
nucl_arr_squ = [6755061, 82975, 4347879, 4280405, 5994621]
nucl_arr_sep_squ = [nucl_arr_sep[i] + nucl_arr_squ[i] for i in range(5)]
nucl_arr_bim_oct = [nucl_arr_bim[i] + nucl_arr_oct[i] for i in range(5)]

nucl_arr_sep_squ_syn = [14537832, 73094, 9243885, 9091674, 12985324]
nucl_arr_sep_squ_nsyn = [14470409, 140517, 9243885, 9091674, 12985324]

bim_n = subst_mat_normalize(smat_bim)
oct_n = subst_mat_normalize(smat_oct)
sep_n = subst_mat_normalize(smat_sep)
squ_n = subst_mat_normalize(smat_squ)
squ_sep_n = subst_mat_normalize(smat_squ_sep)
bim_oct_n = subst_mat_normalize(smat_bim_oct)
squ_sep_syn_n = subst_mat_normalize(smat_squ_sep_syn)
squ_sep_nsyn_n = subst_mat_normalize(smat_squ_sep_nsyn)


print_evol(squ_sep_syn_n, nucl_arr_sep_squ_syn, 50, "squ_sep_evol_syn.txt", "squ_sep_fromE_syn.txt", "squ_sep_toE_syn.txt")
print_evol(squ_sep_nsyn_n, nucl_arr_sep_squ_nsyn, 50, "squ_sep_evol_nsyn.txt", "squ_sep_fromE_nsyn.txt", "squ_sep_toE_nsyn.txt")

print_evol(bim_oct_n, nucl_arr_bim_oct, 500, "bim_oct_evol.txt", "bim_oct_fromE.txt", "bim_oct_toE.txt")
#print_evol(bim_n, nucl_arr_bim, 500, "bim_evol.txt", "bim_fromE.txt", "bim_toE.txt")
#print_evol(oct_n, nucl_arr_oct, 500, "oct_evol.txt", "oct_fromE.txt", "oct_toE.txt")
#print_evol(sep_n, nucl_arr_sep, 50, "sep_evol.txt", "sep_fromE.txt", "sep_toE.txt")
#print_evol(squ_n, nucl_arr_squ, 50, "squ_evol.txt", "squ_fromE.txt", "squ_toE.txt")

In [46]:
def print_evol_control(smat_n, nucl_arr, n_range, outfile_name):
    let_arr = ['A','G','C','T']
    a = np.array([[smat_n[l1][l2] for l2 in let_arr] for l1 in let_arr])
    b = np.array(nucl_arr)
    with open(outfile_name, "w") as outfile:
        outfile.write('\t'.join(let_arr) + '\tS' + '\n')
        n = map(str, list(b))
        outfile.write('\t'.join(map(str, list(b))) + '\t'+ str(sum(b)) + '\n')
        for i in range(n_range):
            b1 = np.matmul(b, a)
            outfile.write('\t'.join(map(str,b1)) + '\t' + str(sum(b1)) + '\n')
            b = b1

def colsum2(smat, let):
#    print(smat)
    let_arr = ['A','G','C','T']
    _sum = 0
    for c in let_arr:
        _sum += smat[let][c]
    return _sum

def print_control(nucl_arr, smat, mult_num, outfile_name):
#    print(nucl_arr)
    new_nucl_arr = nucl_arr[:1] + nucl_arr[2:]
    smat_c = dict()
    for i in ['A','G','C','T']:
        smat_c[i] = dict()
        for j in ['A','G','C','T']:
            smat_c[i][j] = smat[i][j]
    n = dict()
    for k1 in smat_c.keys():
        n[k1] = dict()
        for k2 in smat_c[k1].keys():
            n[k1][k2] = float(smat_c[k1][k2])/colsum2(smat_c, k1)
    print_evol_control(n, new_nucl_arr, mult_num, outfile_name)

In [47]:
#print_control(nucl_arr_sep_squ_syn, smat_squ_sep_syn, 50, "squ_sep_evol_syn_control.txt")
#print_control(nucl_arr_sep_squ_nsyn, smat_squ_sep_nsyn, 50, "squ_sep_evol_nsyn_control.txt")

#print_control(nucl_arr_bim, smat_bim, 500, "bim_evol_control.txt")
#print_control(nucl_arr_oct, smat_oct, 500, "oct_evol_control.txt")
#print_control(nucl_arr_sep, smat_sep, 50, "sep_evol_control.txt")
#print_control(nucl_arr_squ, smat_squ, 50, "squ_evol_control.txt")

nucl_arr_sep_squ = [14397315, 213611, 9243885, 9091674, 12985324]

print_evol(squ_sep_n, nucl_arr_sep_squ, 50, "test.txt", "test_fromE.txt", "test_toE.txt")
print_control(nucl_arr_sep_squ, smat_squ_sep, 50, "test_control.txt")

[14397315, 213611, 9243885, 9091674, 12985324]


In [38]:
def confints_s(smat, precision, alpha_arr):
    sum_A = colsum(smat_squ_sep, 'A')
    sum_E = colsum(smat_squ_sep, 'E')
    sum_G = colsum(smat_squ_sep, 'G')
    
    p_A2E = smat['A']['E']/sum_A
    p_E2G = smat['E']['G']/sum_E
    p_G2A = smat['G']['A']/sum_G
    
    p_A2G = smat['A']['G']/sum_A
    p_E2A = smat['E']['A']/sum_E
    p_G2E = smat['G']['E']/sum_G
    
    mean = (p_A2E * p_E2G * p_G2A)/(p_A2G * p_E2A * p_G2E)
    
    A2E = rand_arr_generate(precision, sum_A, p_A2E)
    E2G = rand_arr_generate(precision, sum_E, p_E2G)
    G2A = rand_arr_generate(precision, sum_G, p_G2A)
    
    A2G = rand_arr_generate(precision, sum_A, p_A2G)
    E2A = rand_arr_generate(precision, sum_E, p_E2A)
    G2E = rand_arr_generate(precision, sum_G, p_G2E)
    
    s_arr = []
    
    for i in range(precision):
        s = (A2E[i] * E2G[i] * G2A[i])/(A2G[i] * E2A[i] * G2E[i])
        s_arr.append(s)
    
    s_arr = sorted(s_arr)
    
    alpha_dict = dict()
    for alpha in alpha_arr:
        alpha_dict[alpha] = dict()
        alpha_dict[alpha]["min"] = s_arr[int(precision*(alpha/2))]
        alpha_dict[alpha]["max"] = s_arr[-int(precision*(alpha/2))]
    
    return mean, alpha_dict

def info_string(mean, alpha_dict, alpha_arr):
    out_s = []
    out_s.append(mean)
    for alpha in alpha_arr:
        out_s.append(alpha_dict[alpha]["min"])
        out_s.append(alpha_dict[alpha]["max"])
    return "\t".join(map(str, out_s))

#p_A2E = smat_squ_sep['A']['E']/colsum(smat_squ_sep, 'A')
#p_E2G = smat_squ_sep['E']['G']/colsum(smat_squ_sep, 'E')
#p_A2G = smat_squ_sep['A']['G']/colsum(smat_squ_sep, 'A')
#t1 = p_A2E*p_E2G/p_A2G
#p_E2A = smat_squ_sep['E']['A']/colsum(smat_squ_sep, 'E')
#p_G2E = smat_squ_sep['G']['E']/colsum(smat_squ_sep, 'G')
#p_G2A = smat_squ_sep['G']['A']/colsum(smat_squ_sep, 'G')
#t2 = p_E2A*p_G2E/p_G2A
#print(t1/t2)

In [40]:
alpha_arr = [0.1, 0.05, 0.01]
precision = 100000
mean, alpha_dict = confints_s(smat_squ_sep, precision, alpha_arr)
print(info_string(mean, alpha_dict, alpha_arr))

mean, alpha_dict = confints_s(smat_squ_sep_syn, precision, alpha_arr)
print(info_string(mean, alpha_dict, alpha_arr))

mean, alpha_dict = confints_s(smat_squ_sep_nsyn, precision, alpha_arr)
print(info_string(mean, alpha_dict, alpha_arr))

2.396528879715465	1.953490110468723	2.8968766919813915	1.8743425505551443	2.999376499057548	1.729082436209287	3.2044693456760025
1.146232535592408	0.8528030869701642	1.486025371211081	0.8041925376472042	1.5583736035731186	0.7104157633824025	1.7091871652409043
7.0608605659339325	5.089423460407033	9.631816655952953	4.762990691172163	10.233212896524579	4.178411020072537	11.52434440412048


In [ ]:
print(9)

In [ ]:
a = 1